In [ ]:
import os

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras import layers
from keras import Input

tf.__version__

In [ ]:
# TODO there is a numpy util to get files from within tgz's and zips
##!curl -s 'https://storage.googleapis.com/0x19f.com/media/kaggle-mlb-pitch-data-2015-2018.tgz' | tar xz

BASEBALL_FILES_BASE = os.getcwd() + "/kaggle-mlb-pitch-data-2015-2018/"
BASEBALL_FILES_BASE = BASEBALL_FILES_BASE + "2019_"  # just look at 2019 to speed things up


pitches = pd.read_csv(BASEBALL_FILES_BASE + 'pitches.csv')
atbats = pd.read_csv(BASEBALL_FILES_BASE + 'atbats.csv')

df = pd.merge(pitches, atbats, how='inner', on='ab_id')

df = df[[
    # situation stuff
    "ab_id", "inning", "top", "outs", "on_1b", "on_2b", "on_3b",

    # matchup
    "pitcher_id", "p_throws", "batter_id", "stand",

    # specific pitch state
    "pitch_num", "b_count", "s_count",
    # TODO need to add running total of pitches for the current pitcher, and maybe strike percentage
    # TODO add the previous n pitches before and their code outcome
    # 
    
    # this is the label we're trying to predict
    "code",
    "pitch_type",
    # TODO also make the label have the location. not just FF (fourseam fastball, but also high inside)
]]

# remove anything without values, hard to have defaults
# df = df.dropna(subset=['pitch_type', 'code'])
df = df.dropna()

## this is our current target label TODO make this categorical 
df["is_fastball"] = np.where(df["pitch_type"].isin(["FC","FF","FT",]), 1, 0)

# get appropriate types. 
# TODO prob can use a schema on the CSV read to get only the columns we're after, alias them to be more
#      useful, and cast to the right type
# TODO consider to_ordinal for cumulative things maybe? inning, outs, pitch_num, b_count, s_count

# TODO there is a numpy util to get files from within tgz's and zips
##!curl -s 'https://storage.googleapis.com/0x19f.com/media/kaggle-mlb-pitch-data-2015-2018.tgz' | tar xz

BASEBALL_FILES_BASE = os.getcwd() + "/kaggle-mlb-pitch-data-2015-2018/"
BASEBALL_FILES_BASE = BASEBALL_FILES_BASE + "2019_"  # just look at 2019 to speed things up


pitches = pd.read_csv(BASEBALL_FILES_BASE + 'pitches.csv')
atbats = pd.read_csv(BASEBALL_FILES_BASE + 'atbats.csv')

df = pd.merge(pitches, atbats, how='inner', on='ab_id')

df = df[[
    # situation stuff
    "ab_id", "inning", "top", "outs", "on_1b", "on_2b", "on_3b",

    # matchup
    "pitcher_id", "p_throws", "batter_id", "stand",

    # specific pitch state
    "pitch_num", "b_count", "s_count",
    # TODO p0: multi hot encode the count
    # TODO need to add running total of pitches for the current pitcher, and maybe strike percentage
    # TODO add the previous n pitches before and their code outcome
    # 
    
    # this is the label we're trying to predict
    "code",
    "pitch_type",
    # TODO also make the label have the location. not just FF (fourseam fastball, but also high inside)
]]

# remove anything without values, hard to have defaults
# df = df.dropna(subset=['pitch_type', 'code'])
df = df.dropna()

## this is our current target label TODO make this categorical 
df["is_fastball"] = np.where(df["pitch_type"].isin(["FC","FF","FT",]), 1, 0)

# get appropriate types. 
# TODO prob can use a schema on the CSV read to get only the columns we're after, alias them to be more
#      useful, and cast to the right type
# TODO consider to_ordinal for cumulative things maybe? inning, outs, pitch_num, b_count, s_count


# TODO consider using ints and normalizing them instead of floats for innnigs etc

df = df.astype({
    'p_throws': 'string',
    'stand': 'string',

    ## these are categorical, treat as strings for now
    'inning': 'string',
    'top': 'string',
    'outs': 'string',
    'on_1b': 'string',
    'on_2b': 'string',
    'on_3b': 'string',
    'pitch_num': 'string',
    'b_count': 'string',
    's_count': 'string',
  })


# drop ab_id now that join has occured, not a useful feature
df = df.drop(['ab_id', "code","pitch_type"], axis=1)

# TODO remove this truncation when solid
df = df.truncate(after=50000)

print(df.info())
print(df.shape)

df.head()


In [ ]:

pitch_features = df.copy()
pitch_labels = pitch_features.pop('is_fastball')

inputs = {}

for name, col in pitch_features.items():
  if str(col.dtype) == "string":
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

In [ ]:
numeric_inputs = {
  name:input for name,input in inputs.items() if input.dtype==tf.float32
}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(pitch_features[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

preprocessed_inputs = [all_numeric_inputs]

for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue

  print("processing %s" % name)
  
  lookup = layers.StringLookup(vocabulary=np.unique(pitch_features[name]))
  one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)


preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

pitch_prep_model = tf.keras.Model(inputs, preprocessed_inputs_cat)


In [ ]:
# tf.keras.utils.plot_model(model = pitch_preprocessing , rankdir="LR", dpi=72, show_shapes=True)


In [ ]:
pitch_features_dict = {name: np.array(value) 
                         for name, value in pitch_features.items()}


def gen_pitch_model(preprocessing_head, inputs):
  body = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1)
  ])

  preprocessed_inputs = preprocessing_head(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile("adam", "binary_crossentropy", metrics=["accuracy"])


  # model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
  #               optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])


  return model

pitch_model = gen_pitch_model(pitch_prep_model, inputs)
pitch_model.fit(x=pitch_features_dict, y=pitch_labels, epochs=30, batch_size=50*1000)



